In [8]:
import os
import rasterio as ras
from osgeo import gdal, ogr
import matplotlib.pyplot as plt
from rasterio.plot import show
import numpy as np
import pandas as pd
import geopandas as gp
from scipy.interpolate import interp1d

In [2]:
ra = ras.open('/home/robert/workbank_2021/africa_risk/Africa/res/landslides/landslides.tif')
shp = '/home/robert/workbank_2021/africa_shp/afr_g2014_2013_0.shp'
fragility = pd.read_csv('/home/robert/workbank_2021/africa_risk/Fragility_curves.csv', header=[0,1], index_col=0)

def create_tifs(files, bounds, res):
    for a in files:
        print(a)
        t = gdal.Open(a)
        end = a.split('/')[-1][:-4]
        gdal.Warp('test_'+end+'.tif', [a], creationOptions=["COMPRESS=DEFLATE"], 
        outputBounds=bounds, xRes=res, yRes=res, dstSRS='EPSG:4326', cutlineDSName=shp)

def calc_intersection(files: list, base_cutoff=0., field_cutoff=[0]):
    gdal_list = []
    for a in files:
        mp = gdal.Open(a)
        x = mp.GetRasterBand(1).ReadAsArray().flatten()
        gdal_list.append(x)
    t = np.where(gdal_list[0] > base_cutoff)
    for i in range(len(gdal_list)):
        y = gdal_list[i][t]
        n = np.where(y > field_cutoff[i])
        print(len(n[0])/len(t[0]))
    return gdal_list

def cutoff_calc(risk_file: str, infrastructure_file: str, risk_cutoff=0.):
    r_array = gdal.Open(risk_file).getRasterBand(1).ReadAsArray().flatten()
    i_array = gdal.Open(infrastructure_file).getRasterBand(1).ReadAsArray().flatten()
    t = np.where(r_array > risk_cutoff)
    n = np.where(y > field_cutoff[i])
    print(len(n[0])/len(t[0]))
    return 

def fragility_calc(file: str, df: object):
    mp = gdal.Open(file)
    arr = mp.GetRasterBand(1).ReadAsArray()
    x = list(df['x'])
    y = list(df['y'])
    f = interp1d(x, y)
    new = f(arr)
    return new

def risk_overlay(risk_file: str, infrastructure_file: str, risk_cutoff: float):
    r_array = gdal.Open(risk_file).getRasterBand(1).ReadAsArray()
    i_array = gdal.Open(infrastructure_file).getRasterBand(1).ReadAsArray()
    for i in range(len(r_array)):
        for j in range(len(r_array[i])):
            if r_array[i][j] <= risk_cutoff:
                i_array[i][j] = np.nan
    return i_array

def write_tif(out_file: str, geo_tif_in: str, input_array, band):
    driver = gdal.GetDriverByName("GTiff")
    geotrans = gdal.Open(geo_tif_in)
    out_ds = driver.Create(out_file, input_array.shape[1], input_array.shape[0], 1, gdal.GDT_Float32)
    out_ds.SetProjection(geotrans.GetProjection())
    out_ds.SetGeoTransform(geotrans.GetGeoTransform())
    band = out_ds.GetRasterBand(1)
    band.WriteArray(input_array)
    band.FlushCache()
    band.ComputeStatistics(False)


In [3]:
in_files = [#'/home/robert/workbank_2021/africa_risk/powerplants.tif',
        '/home/robert/workbank_2021/africa_risk/gridfind.tif',
        '/home/robert/workbank_2021/africa_risk/Africa/res/landslides/landslides.tif',
        '/home/robert/workbank_2021/africa_risk/Africa/res/flooding/flooding.tif',
        '/home/robert/workbank_2021/africa_risk/Africa/res/fire/fire.tif',
        '/home/robert/workbank_2021/africa_risk/Africa/res/seismic/pga_475y.tif',
        '/home/robert/workbank_2021/africa_risk/lv.tif'
        ]

out_files = [#'/home/robert/workbank_2021/africa_risk/test_powerplants.tif',
        '/home/robert/workbank_2021/africa_risk/test_gridfind.tif',
        '/home/robert/workbank_2021/africa_risk/test_flooding.tif',
        '/home/robert/workbank_2021/africa_risk/test_landslides.tif',
        '/home/robert/workbank_2021/africa_risk/test_fire.tif',
        '/home/robert/workbank_2021/africa_risk/test_pga_475y.tif',
        '/home/robert/workbank_2021/africa_risk/test_lv.tif'
        ]

field_cutoff = [0, 0, 0, 0, 0, 0]
create_tifs(in_files, ra.bounds, ra.res[0])
print('tifs done')
l1 = calc_intersection(out_files, field_cutoff=field_cutoff)

/home/robert/workbank_2021/africa_risk/gridfind.tif
/home/robert/workbank_2021/africa_risk/Africa/res/landslides/landslides.tif


In [ ]:
def Feature_to_Raster(input_shp, output_tiff, cellsize, field_name=False, NoData_value=-9999):
    """
    Converts a shapefile into a raster
    """

    # Input
    inp_driver = ogr.GetDriverByName('ESRI Shapefile')
    inp_source = inp_driver.Open(input_shp, 0)
    inp_lyr = inp_source.GetLayer()
    inp_srs = inp_lyr.GetSpatialRef()

    # Extent
    x_min, x_max, y_min, y_max = inp_lyr.GetExtent()
    x_ncells = int((x_max - x_min) / cellsize)
    y_ncells = int((y_max - y_min) / cellsize)

    # Output
    out_driver = gdal.GetDriverByName('GTiff')
    out_source = out_driver.Create(output_tiff, x_ncells, y_ncells,1, gdal.GDT_Int16)
    print("+"*50)
    print(x_ncells, y_ncells,1, gdal.GDT_Int16)

    out_source.SetGeoTransform((x_min, cellsize, 0, y_max, 0, -cellsize))
    out_source.SetProjection(inp_srs.ExportToWkt())
    out_lyr = out_source.GetRasterBand(1)
    out_lyr.SetNoDataValue(NoData_value)

    # Rasterize
    # print(inp_lyr)
    if field_name:
        gdal.RasterizeLayer(out_source, [1], inp_lyr,options=["ATTRIBUTE={0}".format(field_name)])
    else:
        gdal.RasterizeLayer(out_source, [1], inp_lyr, burn_values=[1])

    # Save and/or close the data sources
    inp_source = None
    out_source = None

    # Return
    return output_tiff

In [ ]:
#Feature_to_Raster('openstrmap/openstrmap.shp', 'openstrmap.tif', ra.res[0])
#Feature_to_Raster('gridfind/gridfind.shp', 'gridfind.tif', ra.res[0])

In [ ]:
some = gdal.Open('/home/robert/workbank_2021/africa_risk/test_flooding.tif')
some.GetRasterBand(1).ReadAsArray()[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
n = fragility_calc('/home/robert/workbank_2021/africa_risk/test_flooding.tif', fragility['TD Flood'])

TypeError: object of type 'int' has no len()

In [3]:
gdal.Warp('test_moz.tif', '/home/robert/workbank_2021/africa_risk/Flood Maps/africa_flood_lv_risk_0ft.tif', cutlineDSName='/home/robert/workbank_2021/africa_shp/country_shapes/Mozambique.shp', cropToCutline=True)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f2d137f53f0> >

In [17]:
afr = gp.read_file('/home/robert/workbank_2021/africa_shp/afr_g2014_2013_0.shp')
ctr = afr['ADM0_NAME'].unique()
path_i = '/home/robert/workbank_2021/africa_risk/Flood_Maps/country_maps/'
shp_path = '/home/robert/workbank_2021/africa_shp/country_shapes/'
for c in ctr:
        outfile = path_i + c +'/' + c + '.tif'
        gtif_file = '/home/robert/workbank_2021/africa_risk/Flood_Maps/africa_flood_risk.tif'
        shp_file = shp_path + c + '.shp'
        gdal.Warp(outfile, 
                gtif_file, 
                cutlineDSName=shp_file, 
                cropToCutline=True)


In [12]:
path = '/home/robert/workbank_2021/africa_risk/Flood_Maps/africa_flood_lv_risk_'
fi_str = ''
for i in range(0, 24):
    temp = path + str(i) + 'ft.tif'
    fi_str += temp
    fi_str += ' '

import os
command = "gdal_merge.py -separate -o /home/robert/workbank_2021/africa_risk/Flood_Maps/africa_flood_risk.tif -of gtiff " + fi_str
print(os.popen(command).read())

0...10...20...30...40...50...60...70...80...90...100 - done.



In [13]:
pfs = gdal.Open('/home/robert/workbank_2021/africa_risk/Flood_Maps/africa_flood_risk.tif')

In [14]:
pfs.RasterCount

24